In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


### Read data

In [2]:
#import nltk
#nltk.download('popular')

In [3]:
data = pd.read_excel("trining set.xlsx")

In [4]:
data.head()

,Unnamed: 0,name,text,Stance
0,0,مشاعل #الزعيم_العالمي_الملكي,إعلان واطي من سكشن-بي 😐😑😏 https://t.co/uKArk6lg5k,neg
1,1,Abureem,@shrimpshacksa للأسف كانت تجربة سيئة معكم امس ...,neg
2,2,ُBo riyad ... ♚,RT @Mur_Hus: @Saad_Maestro @MaestroPizzaKSA @l...,neg
3,3,إبراهيم الملحم,@SubwaySaudi آخر مرة بإذن الله اشتري منكم ، كل...,neg
4,4,Different,اليوم بروح سكشن بي منقبة عناد,neg


In [5]:
data=data[['name','text','Stance']]


In [6]:
data

,name,text,Stance
0,مشاعل #الزعيم_العالمي_الملكي,إعلان واطي من سكشن-بي 😐😑😏 https://t.co/uKArk6lg5k,neg
1,Abureem,@shrimpshacksa للأسف كانت تجربة سيئة معكم امس ...,neg
2,ُBo riyad ... ♚,RT @Mur_Hus: @Saad_Maestro @MaestroPizzaKSA @l...,neg
3,إبراهيم الملحم,@SubwaySaudi آخر مرة بإذن الله اشتري منكم ، كل...,neg
4,Different,اليوم بروح سكشن بي منقبة عناد,neg
...,...,...,...
3738,adel 22,جميل المطعم بي الزحمة تكرهني فيه .. والله تعقد...,neg
3739,Fares Aljumaid,الشاورما ⭐️⭐️⭐️⭐️⭐️الكباب ⭐️⭐️⭐️⭐️⭐️أوصال الدج...,pos
3740,M,لاتوجد قهوه ضمن الافطار ! فقط عصيرات عالية الس...,neg
3741,shahd mohamad,الشورما و فطيرة الجبن 👌🏼,pos


In [7]:
data.shape

(3743, 3)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    3743 non-null   object
 1   text    3743 non-null   object
 2   Stance  3743 non-null   object
dtypes: object(3)
memory usage: 87.9+ KB


In [9]:
data=data[data['Stance']!='neu']

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1904 entries, 0 to 3742
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    1904 non-null   object
 1   text    1904 non-null   object
 2   Stance  1904 non-null   object
dtypes: object(3)
memory usage: 59.5+ KB


In [11]:
data.Stance=data.Stance.map({'neg':0,'pos':1})

C:\Users\ibrahim\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
data

,name,text,Stance
0,مشاعل #الزعيم_العالمي_الملكي,إعلان واطي من سكشن-بي 😐😑😏 https://t.co/uKArk6lg5k,0
1,Abureem,@shrimpshacksa للأسف كانت تجربة سيئة معكم امس ...,0
2,ُBo riyad ... ♚,RT @Mur_Hus: @Saad_Maestro @MaestroPizzaKSA @l...,0
3,إبراهيم الملحم,@SubwaySaudi آخر مرة بإذن الله اشتري منكم ، كل...,0
4,Different,اليوم بروح سكشن بي منقبة عناد,0
...,...,...,...
3738,adel 22,جميل المطعم بي الزحمة تكرهني فيه .. والله تعقد...,0
3739,Fares Aljumaid,الشاورما ⭐️⭐️⭐️⭐️⭐️الكباب ⭐️⭐️⭐️⭐️⭐️أوصال الدج...,1
3740,M,لاتوجد قهوه ضمن الافطار ! فقط عصيرات عالية الس...,0
3741,shahd mohamad,الشورما و فطيرة الجبن 👌🏼,1


In [13]:
# for i in data.cla:
#     print(i)
#     break

In [14]:
data.Stance.unique()

array([0, 1], dtype=int64)

In [15]:
data.shape

(1904, 3)

In [16]:
data.Stance.value_counts()

1    952
0    952
Name: Stance, dtype: int64

In [17]:
pos = []
neg = []
for l in data.Stance:

    if l == 1:
        pos.append(1)
        neg.append(0)
        
    elif l == 0: 
        pos.append(0)
        neg.append(1)
    
        
        

In [18]:
len(pos)

1904

In [19]:
data['Pos']= pos
data['Neg']= neg


C:\Users\ibrahim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\ibrahim\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
data.head()

,name,text,Stance,Pos,Neg
0,مشاعل #الزعيم_العالمي_الملكي,إعلان واطي من سكشن-بي 😐😑😏 https://t.co/uKArk6lg5k,0,0,1
1,Abureem,@shrimpshacksa للأسف كانت تجربة سيئة معكم امس ...,0,0,1
2,ُBo riyad ... ♚,RT @Mur_Hus: @Saad_Maestro @MaestroPizzaKSA @l...,0,0,1
3,إبراهيم الملحم,@SubwaySaudi آخر مرة بإذن الله اشتري منكم ، كل...,0,0,1
4,Different,اليوم بروح سكشن بي منقبة عناد,0,0,1


### Clean data

In [21]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['text'].apply(lambda x: remove_punct(x))

C:\Users\ibrahim\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean] 

In [23]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [24]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [25]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [26]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [27]:
result = [' '.join(sen) for sen in filtered_words] 

In [28]:
data['Text_Final'] = result

C:\Users\ibrahim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
data['tokens'] = filtered_words

C:\Users\ibrahim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
data = data[['Text_Final', 'tokens', 'Stance', 'Pos', 'Neg']]

In [31]:
data[:4]

,Text_Final,tokens,Stance,Pos,Neg
0,إعلان واطي من سكشنبي 😐😑😏 httpstcoukark6lg5k,"[إعلان, واطي, من, سكشنبي, 😐😑😏, httpstcoukark6l...",0,0,1
1,shrimpshacksa للأسف كانت تجربة سيئة معكم امس و...,"[shrimpshacksa, للأسف, كانت, تجربة, سيئة, معكم...",0,0,1
2,rt murhus saadmaestro maestropizzaksa lcsaudi ...,"[rt, murhus, saadmaestro, maestropizzaksa, lcs...",0,0,1
3,subwaysaudi آخر مرة بإذن الله اشتري منكم ، كل ...,"[subwaysaudi, آخر, مرة, بإذن, الله, اشتري, منك...",0,0,1


### Split data into test and train

In [32]:
data_train, data_test = train_test_split(data, test_size=0.50, random_state=42)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1904 entries, 0 to 3742
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Text_Final  1904 non-null   object
 1   tokens      1904 non-null   object
 2   Stance      1904 non-null   int64 
 3   Pos         1904 non-null   int64 
 4   Neg         1904 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 89.2+ KB


In [34]:
len(data_train)

952

In [35]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

10229 words total, with a vocabulary size of 3821
Max sentence length is 37


In [36]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

10064 words total, with a vocabulary size of 3791
Max sentence length is 29


### Load Google News Word2Vec model

In [37]:
word2vec_path = r'Word2Vec_Model.bin'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [38]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

### Get Embeddings

In [39]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [40]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

### Tokenize and Pad sequences

In [41]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 3821 unique tokens.


In [42]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [43]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(3822, 300)


In [44]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Define CNN

In [45]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    #The embeddings matrix is passed to embedding_layer
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        #Aggregation 5 filter into one value
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [46]:
label_names = ['Pos', 'Neg']

In [47]:
y_train = data_train[label_names].values

In [48]:
x_train = train_cnn_data
y_tr = y_train

In [49]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 300)      1146600     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 49, 200)      120200      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 48, 200)      180200      embedding_1[0][0]                
____________________________________________________________________________________________

### Train CNN

In [50]:
num_epochs = 5
batch_size = 1

In [51]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 856 samples, validate on 96 samples
Epoch 1/5
856/856 [==============================] - 11s 13ms/step - loss: 0.6270 - acc: 0.7231 - val_loss: 0.5427 - val_acc: 0.7448
Epoch 2/5
856/856 [==============================] - 11s 13ms/step - loss: 0.4554 - acc: 0.8201 - val_loss: 0.4418 - val_acc: 0.8073
Epoch 3/5
856/856 [==============================] - 11s 13ms/step - loss: 0.3295 - acc: 0.8610 - val_loss: 0.4227 - val_acc: 0.8333
Epoch 4/5
856/856 [==============================] - 11s 13ms/step - loss: 0.2754 - acc: 0.8820 - val_loss: 0.5825 - val_acc: 0.7969
Epoch 5/5
856/856 [==============================] - 11s 13ms/step - loss: 0.1997 - acc: 0.9264 - val_loss: 0.6804 - val_acc: 0.8177


### Test CNN

In [52]:
predictions = model.predict(test_cnn_data, batch_size=1, verbose=1)

952/952 [==============================] - 1s 1ms/step


In [53]:
labels = [1, 0]

In [58]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [59]:
prediction_labels

[1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,


In [60]:
sum(data_test.Stance==prediction_labels)/len(prediction_labels)

0.8308823529411765

In [61]:
data_test.Stance.value_counts()

0    478
1    474
Name: Stance, dtype: int64